# Connect DB (SQLite) to Airflow

## Training Objective

This notebook will perform how to connect and combine SQLite, a lightweight and efficient relational database, with Airflow, open source platform for worfklow automation and scheduling.

Integrating SQLite with Airflow allows us to interact with our SQLite database, run quesries and load or export data from an Airflow DAG.

- **Introduction to SQLite**
  - SQLite Definition
  - Define SQLite in Python and Airflow
- **Connect SQLite to Airflow**
  - Creating DAG to fetch youtube API, Preprocess the data, and load to SQLite database.
  - Creating task sequence and scheduling in DAG.

## Library Preparation

In [2]:
# import necessary libraries
import sqlite3 # sqlite3 database
import pandas as pd # pandas for dataframe

# Introduction to SQLite

## SQLite Definition

SQLite is one of the most popular and easy-to-use relational database. It is an open-source library with zero configuration and does not require installation.

Why use SQLite:

- SQLite is an open-source software. The software does not require any license after installation.
- SQLite is serverless as it doesn't need a different server process or system to operate.
- SQLite facilitates you to work on multiple databases on the same session simultaneously, thus making it flexible.
- SQLite is a cross-platform DBMS that can run on all platforms, including macOS, Windows, etc.
- SQLite doesn't require any configuration. It needs no setup or administration.

For these reasons we use SQLite as our database project since it is really easy to config and doesn't need any installation to use it with Python.

*Note: For new airflow project that will use SQLite as database and want to run it in Docker. Please use `docker-compose.yaml` from this project that has configure for SQLite database in airflow.

## Define SQLite in Python and Airflow

### SQLite Database Path in Airflow

First thing to ensure is the path of SQLite database stored in Airflow project. To make sure/modify the path:

- Open the `docker-compose.yaml`.
- Search `AIRFLOW__DATABASE__SQL_ALCHEMY_CONN`. It stores the path of SQLite database.
- The path of the SQLite database should be `sqlite:////opt/airflow/db/airflow.db`. So if we want to create SQLite database, it should be in `db/airflow.db` in project folder or `/opt/airflow/db/airflow.db` in Docker Airflow project.

### Create SQLite Database with Python

Creating SQLite database can be performed in python using `sqlite3` built-in library (no need installation).

The steps to creating database in python:

- Import `sqlite3` library
- Create python connection to SQLite database with
  ```python
  conn = sqlite3.connect("database path")
  ```

If the database already exist, than python will connect with existing database.

Example to creating new SQLite database in Python

In [3]:
db_filepath = 'db/trial.db'
conn = sqlite3.connect(db_filepath)

This code will create database named `trial.db` (if didn't exist) in `db` folder.

### Insert to Database from DataFrame

`pandas` has provide method to insert dataframe directly into database. The method To insert dataframe into database is `df.to_sql()`. The needed input is the name of the target table and the dataframe itself, the other inputs can be adjusted as needed preference.

```python
df.to_sql(name,con,if_exist,index)
```

**Parameter:**

- `name`: Name of SQL table
- `con`: Connection object
- `if_exist`: How to behave if table already exist.
  - `fail`: Raise a ValueError
  - `replace`: Drop the table before inserting new values
  - `append`: Insert new values to the existing table.
- `index`: Write Dataframe index as a column.

For example we have dataframe below and want to insert into `users` table in `trial.db` database.

In [4]:
df = pd.DataFrame({'name' : ['User 1', 'User 2', 'User 3']})
df

,name
0,User 1
1,User 2
2,User 3


In [9]:
# insert into users table
df.to_sql(name='users', con=conn, if_exists='append')

3

The method will return the number of data successfully inserted.

### Look into the Table

There are 2 ways to show our data in the database table.
1. `pandas` way: we can use pandas to query all the data from certain table
2. SQLite viewer way: we can use `SQLite` extension from VSCode to look dive into our data in database.

We will breakdown briefly each way in this section.

#### 1. Pandas way

To get (commonly called `fetch`) the data from database, we can utilize the `conn` object we have created that connect python with the database.

The method to fetch data using pandas from database is `pd.read_sql_query(sql, con)` that need query and connection object as input.

**Parameter:**
- `sql`: SQL Query
- `con`: Connection object

Let's fetch all data in our `users` table in `trial.db` database.

In [10]:
pd.read_sql_query(sql = "SELECT * FROM USERS",
                con = conn)

,index,name
0,0,User 1
1,1,User 2
2,2,User 3


#### 2. SQLite Viewer Way

SQLite Viewer is an extension in VSCode that provide functionality to open and see SQLite database. The steps to use SQLite viewer are:

1. Install SQLite Viewer Extension
   First, search "SQLite Viewer" in Extensions tab and click "install"
   ![](assets/sqlite_viewer.png)
2. Search and double click the target db to open in VSCode Explorer, in this case we want to open `db/trial.db`.
3. It will open new window that show our data within the table.
   ![](assets/trialdb.png)

# Connect SQLite to Airflow

Now, we have know how to connect to database and insert data into it from pandas dataframe. Now let's create functionality to insert processed youtube data in `tmp_file_processed.json` to SQLite database.

First, we need to load our data from the json file and stored it as dataframe.

In [ ]:
source_file_path = 'dags/tmp_file_processed.json'

df = pd.read_json(source_file_path)

df.head()

,videoId,trendingAt,title,publishedAt,channelId,channelTitle,description,tags,defaultAudioLanguage,caption,viewCount,likeCount,commentCount,durationSec,category,thumbnailUrl
0,lRbxVxbbavU,2024-04-19T10:06:41Z,HIGHLIGHTS! Indonesia (1) vs (0) Australia | A...,2024-04-18T15:26:56Z,UCeM5Nksgv9_FXTuZ8jkPJPg,RCTI - ENTERTAINMENT,Permainan yang sangat menegangkan dari kedua N...,"RCTI, Timnas Indonesia, Garuda indonesia, Pial...",id,False,3310552,47123,7503,1202,Sports,https://i.ytimg.com/vi/lRbxVxbbavU/sddefault.jpg
1,bCZZ7zyGtYE,2024-04-19T10:06:41Z,Manchester City VS Real Madrid | Highlights Li...,2024-04-18T00:00:19Z,UC_vsErcsq56hOscPHkG-aVw,SCTV,Tonton full highlights https://www.vidio.com/w...,"sctv, indonesia, movies, eufa champions league...",id,False,1102265,12179,2554,72,Sports,https://i.ytimg.com/vi/bCZZ7zyGtYE/sddefault.jpg
2,g7pMevhVeSM,2024-04-19T10:06:41Z,HAPPY ASMARA Feat. GILGA SAHID - LAMUNAN | Fea...,2024-04-17T03:00:08Z,UCEdO8A1SpMMF47LuGGV1nMQ,MUARA BINTANG,HAPPY ASMARA Feat. GILGA SAHID - LAMUNAN | Fea...,"indosiar, jtv, sctv, rcti, viral, terpopuler, ...",id,True,1120373,102722,30719,306,Music,https://i.ytimg.com/vi/g7pMevhVeSM/sddefault.jpg
3,BaYJwfPx3oY,2024-04-19T10:06:41Z,HIGHLIGHT QATAR VS INDONESIA| AFC U23 ASIAN CU...,2024-04-15T18:40:43Z,UCE9-bV_MCGgLnH7v4HSApDg,Official RCTI,Ayo saksikan Full Episode program RCTI lainnya...,"RCTI, Official RCTI, Sinetron Terbaru, Film te...",id,False,2280191,14492,7933,1050,Entertainment,https://i.ytimg.com/vi/BaYJwfPx3oY/sddefault.jpg
4,D1ie232_ylQ,2024-04-19T10:06:41Z,1 HARI MENJADI GANGSTER - GTA 5 ROLEPLAY,2024-04-18T00:01:06Z,UCqPN9LAkx20aBWOafwvQB-Q,Jelool,Subscribe ► https://www.youtube.com/c/Jelool\n...,"executive roleplay, misi gta 5, balap liar gta...",id,False,415469,39132,7887,1987,Gaming,https://i.ytimg.com/vi/D1ie232_ylQ/sddefault.jpg


After we read the json file. We need to load our dataframe to SQLite database. First, let's create or connect to our database, `db/airflow.db` that have been define in `docker-compose.yaml`.

In [ ]:
# create connection
database = 'db/airflow.db'
conn = sqlite3.connect(database)

In this case, we want to insert our `df` dataframe into `trending_videos` table in `db/airflow.db`.

To achieve this goals, we will use `df.to_sql()`.

In [ ]:
# Append the DataFrame to the existing table if it exists, otherwise create a new table
table_name = 'trending_videos'
df.to_sql(name = table_name, con=conn, 
            if_exists='append', 
            index=False)

# Close the connection
conn.close()

Now we have insert clean trending videos data to airflow.db. If we arrange the steps to insert the data to `load_to_sqlite` function, should be look like this.

In [ ]:
def load_to_sqlite(source_file_path: str, table_name: str):
    """
    Loads the processed data to SQLite.
    
    Args:
        source_file_path: A string representing the path to the file to be loaded.
        table_name: A string representing the name of the table to load the data to.
    """
    
    # Load the data from the json file to sqlite
    df = pd.read_json(source_file_path)
    database = "/opt/airflow/db/airflow.db"
    conn = sqlite3.connect(database)
    
    # Append the DataFrame to the existing table if it exists, otherwise create a new table
    df.to_sql(name=table_name, con=conn, if_exists='append', index=False)
    
    conn.close()

    # Log the job results
    print("Done Created DB")

## Creating DAG

Integrating SQLite with Airflow allows us to interact with our SQLite database, run quesries and load or export data from an Airflow DAG.

We need to create these following tasks to run in Airflow DAG:
- `fetch_trending_videos` task, fetch youtube trending videos as we have disscussed in first module `01_fetching_data_api`.
- `data_processing` task, to clean and process fetching result before load it to database SQLite.
- `load_to_sqlite` task, load data to database SQLite after data processing.

We will complete and continue our DAG from `02_airflow_introduction` module. Our current DAG:

```python
@dag(dag_id='trending_youtube_dag_v1',
    default_args=default_args,
    description='A pipeline to fetch trending YouTube videos',
    start_date=datetime(2023, 5, 7, tzinfo=timezone(timedelta(hours=7))),
    schedule_interval='0 10 * * *',
    catchup=False
)
def trending_youtube_dag():
    '''
    This is youtube trending dag, we will define the task in the next section
    '''
    @task()
    def fetch_trending_videos(region_code: str, file_path: str):
        '''
        function to be used for fetching trending videos
        '''
    
    @task()
    def data_processing(source_file_path: str, target_file_path: str):
        '''
        Function to be used for preprocess the data.
        '''
    
    file_path = '/opt/airflow/dags/tmp_file.json'
    fetch_trending_videos_task = fetch_trending_videos(region_code='ID', max_results=200, target_file_path=file_path)
    processed_file_path = '/opt/airflow/dags/tmp_file_processed.json'
    data_processing_task = data_processing(source_file_path=file_path, target_file_path=processed_file_path)
    
    fetch_trending_videos_task >> data_processing_task
    
dag = trending_youtube_dag()
```

### fetch_trending_videos

We need to move our code to fetching trending video in `01_fetching_data_api` module to `fetch_treding_videos` task.

Let's complete the `fetch_trending_videos` task.

In [ ]:
def fetch_trending_videos(region_code: str, file_path: str):
    '''
    function to be used for fetching trending videos
    '''

    # Load API key from .env file
    load_dotenv("/opt/airflow/dags/.env")
    api_key = os.environ.get("YOUTUBE_API_KEY")

    # Create YouTube API client
    youtube = build("youtube", "v3", developerKey=api_key)

    # Fetch videos until max_results is reached or there are no more results
    videos_list = []
    next_page_token = ""
    while len(videos_list) < max_results and next_page_token is not None:
        # Make API request for videos
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            chart="mostPopular",
            regionCode=region_code,
            maxResults=50,
            pageToken=next_page_token,
        )
        response = request.execute()

        # Extract videos from response
        videos = response.get("items", [])

        # Update next_page_token for the next API request
        next_page_token = response.get("nextPageToken", None)
        
        # Extract relevant video details and append to list
        infos = {'snippet':['title', 'publishedAt', 'channelId', 'channelTitle',
                            'description', 'tags', 'thumbnails', 'categoryId', 'defaultAudioLanguage'],
                    'contentDetails':['duration', 'caption'],
                    'statistics':['viewCount', 'likeCount', 'commentCount']}
        now = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
        for video in videos:
            video_details = {
                'videoId': video["id"],
                'trendingAt': now
            }
            
            for k in infos.keys():
                for info in infos[k]:
                    # use try-except to handle missing info
                    try:
                        video_details[info] = video[k][info]
                    except KeyError:
                        video_details[info] = None
                        
            videos_list.append(video_details)

    # Write fetched videos data to a json file
    with open(target_file_path, "w") as f:
        json.dump(videos_list, f)

Now we have create function that perform fetching data from the start until it save certain column/component in json format.

Our updated DAG after completing `fetch_trending_videos` should like this.

In [ ]:
@dag(dag_id='trending_youtube_dag_v1',
    default_args=default_args,
    description='A pipeline to fetch trending YouTube videos',
    start_date=datetime(2023, 5, 7, tzinfo=timezone(timedelta(hours=7))),
    schedule_interval='0 10 * * *',
    catchup=False
)
def trending_youtube_dag():
    '''
    This is youtube trending dag, we will define the task in the next section
    '''
    @task()
    def fetch_trending_videos(region_code: str, file_path: str):
        '''
        function to be used for fetching trending videos
        '''

        # Load API key from .env file
        load_dotenv("/opt/airflow/dags/.env")
        api_key = os.environ.get("YOUTUBE_API_KEY")

        # Create YouTube API client
        youtube = build("youtube", "v3", developerKey=api_key)

        # Fetch videos until max_results is reached or there are no more results
        videos_list = []
        next_page_token = ""
        while len(videos_list) < max_results and next_page_token is not None:
            # Make API request for videos
            request = youtube.videos().list(
                part="snippet,contentDetails,statistics",
                chart="mostPopular",
                regionCode=region_code,
                maxResults=50,
                pageToken=next_page_token,
            )
            response = request.execute()

            # Extract videos from response
            videos = response.get("items", [])

            # Update next_page_token for the next API request
            next_page_token = response.get("nextPageToken", None)
            
            # Extract relevant video details and append to list
            infos = {'snippet':['title', 'publishedAt', 'channelId', 'channelTitle',
                                'description', 'tags', 'thumbnails', 'categoryId', 'defaultAudioLanguage'],
                        'contentDetails':['duration', 'caption'],
                        'statistics':['viewCount', 'likeCount', 'commentCount']}
            now = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            for video in videos:
                video_details = {
                    'videoId': video["id"],
                    'trendingAt': now
                }
                
                for k in infos.keys():
                    for info in infos[k]:
                        # use try-except to handle missing info
                        try:
                            video_details[info] = video[k][info]
                        except KeyError:
                            video_details[info] = None
                            
                videos_list.append(video_details)

        # Write fetched videos data to a json file
        with open(target_file_path, "w") as f:
            json.dump(videos_list, f)
    
    @task()
    def data_processing(source_file_path: str, target_file_path: str):
        '''
        Function to be used for preprocess the data.
        '''
    
    file_path = '/opt/airflow/dags/tmp_file.json'
    fetch_trending_videos_task = fetch_trending_videos(region_code='ID', max_results=200, target_file_path=file_path)
    processed_file_path = '/opt/airflow/dags/tmp_file_processed.json'
    data_processing_task = data_processing(source_file_path=file_path, target_file_path=processed_file_path)
    
    fetch_trending_videos_task >> data_processing_task
    
dag = trending_youtube_dag()

### data_processing

After completing fetching task, we need to completing and move the data processing code from `01_fetching_data_api` to `trending_youtube_dag_sqlite` DAG.

Let's complete our `data_processing` function.

In [ ]:
def data_processing(source_file_path: str, target_file_path: str):
    
    """Processes the raw data fetched from YouTube.
    Args:
        source_file_path: A string representing the path to the file to be processed.
        target_file_path: A string representing the path to the file to be written.
    """
    # Load the fetched videos data from the json file
    with open(source_file_path, 'r') as f:
        videos_list = json.load(f)
    
    # Load the categories dictionary from the json file
    with open('/opt/airflow/dags/categories.json', 'r') as f:
        categories = json.load(f)
    
    # Process the fetched videos data
    for video in videos_list:
        # Convert ISO 8601 duration to seconds
        video['durationSec'] = int(isodate.parse_duration(video['duration']).total_seconds()) if video['duration'] is not None else None
        del video['duration']
        
        # Convert tags list to string
        video['tags'] = ', '.join(video['tags']) if video['tags'] is not None else None
        
        # Convert categoryId to category based on categories dictionary
        video['category'] = categories.get(video['categoryId'], None) if video['categoryId'] is not None else None
        del video['categoryId']

        # Parse the thumbnail url
        video['thumbnailUrl'] = video['thumbnails'].get('standard', {}).get('url', None) if video['thumbnails'] is not None else None
        del video['thumbnails']
        
        # Convert viewCount, likeCount, and commentCount to integer
        video['viewCount'] = int(video['viewCount']) if video['viewCount'] is not None else None
        video['likeCount'] = int(video['likeCount']) if video['likeCount'] is not None else None
        video['commentCount'] = int(video['commentCount']) if video['commentCount'] is not None else None
        
        # Convert caption to boolean
        video['caption'] = True if video['caption'] == 'true' else False if video['caption'] == 'false' else None
    
    # Save the processed videos data to a new file
    with open(target_file_path, "w") as f:
        json.dump(videos_list, f)

After we complete our `data_processing` function. The updated DAG should be look like below

In [ ]:
@dag(dag_id='trending_youtube_dag_v1',
    default_args=default_args,
    description='A pipeline to fetch trending YouTube videos',
    start_date=datetime(2023, 5, 7, tzinfo=timezone(timedelta(hours=7))),
    schedule_interval='0 10 * * *',
    catchup=False
)
def trending_youtube_dag():
    '''
    This is youtube trending dag, we will define the task in the next section
    '''
    @task()
    def fetch_trending_videos(region_code: str, file_path: str):
        '''
        function to be used for fetching trending videos
        '''

        # Load API key from .env file
        load_dotenv("/opt/airflow/dags/.env")
        api_key = os.environ.get("YOUTUBE_API_KEY")

        # Create YouTube API client
        youtube = build("youtube", "v3", developerKey=api_key)

        # Fetch videos until max_results is reached or there are no more results
        videos_list = []
        next_page_token = ""
        while len(videos_list) < max_results and next_page_token is not None:
            # Make API request for videos
            request = youtube.videos().list(
                part="snippet,contentDetails,statistics",
                chart="mostPopular",
                regionCode=region_code,
                maxResults=50,
                pageToken=next_page_token,
            )
            response = request.execute()

            # Extract videos from response
            videos = response.get("items", [])

            # Update next_page_token for the next API request
            next_page_token = response.get("nextPageToken", None)
            
            # Extract relevant video details and append to list
            infos = {'snippet':['title', 'publishedAt', 'channelId', 'channelTitle',
                                'description', 'tags', 'thumbnails', 'categoryId', 'defaultAudioLanguage'],
                        'contentDetails':['duration', 'caption'],
                        'statistics':['viewCount', 'likeCount', 'commentCount']}
            now = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            for video in videos:
                video_details = {
                    'videoId': video["id"],
                    'trendingAt': now
                }
                
                for k in infos.keys():
                    for info in infos[k]:
                        # use try-except to handle missing info
                        try:
                            video_details[info] = video[k][info]
                        except KeyError:
                            video_details[info] = None
                            
                videos_list.append(video_details)

        # Write fetched videos data to a json file
        with open(target_file_path, "w") as f:
            json.dump(videos_list, f)
    
    @task()
    def data_processing(source_file_path: str, target_file_path: str):
        """Processes the raw data fetched from YouTube.
        
        Args:
            source_file_path: A string representing the path to the file to be processed.
            target_file_path: A string representing the path to the file to be written.
        """
        # Load the fetched videos data from the json file
        with open(source_file_path, 'r') as f:
            videos_list = json.load(f)
        
        # Load the categories dictionary from the json file
        with open('/opt/airflow/dags/categories.json', 'r') as f:
            categories = json.load(f)
        
        # Process the fetched videos data
        for video in videos_list:
            # Convert ISO 8601 duration to seconds
            video['durationSec'] = int(isodate.parse_duration(video['duration']).total_seconds()) if video['duration'] is not None else None
            del video['duration']
            
            # Convert tags list to string
            video['tags'] = ', '.join(video['tags']) if video['tags'] is not None else None
            
            # Convert categoryId to category based on categories dictionary
            video['category'] = categories.get(video['categoryId'], None) if video['categoryId'] is not None else None
            del video['categoryId']

            # Parse the thumbnail url
            video['thumbnailUrl'] = video['thumbnails'].get('standard', {}).get('url', None) if video['thumbnails'] is not None else None
            del video['thumbnails']
            
            # Convert viewCount, likeCount, and commentCount to integer
            video['viewCount'] = int(video['viewCount']) if video['viewCount'] is not None else None
            video['likeCount'] = int(video['likeCount']) if video['likeCount'] is not None else None
            video['commentCount'] = int(video['commentCount']) if video['commentCount'] is not None else None
            
            # Convert caption to boolean
            video['caption'] = True if video['caption'] == 'true' else False if video['caption'] == 'false' else None
        
        # Save the processed videos data to a new file
        with open(target_file_path, "w") as f:
            json.dump(videos_list, f)
    
    file_path = '/opt/airflow/dags/tmp_file.json'
    fetch_trending_videos_task = fetch_trending_videos(region_code='ID', max_results=200, target_file_path=file_path)
    processed_file_path = '/opt/airflow/dags/tmp_file_processed.json'
    data_processing_task = data_processing(source_file_path=file_path, target_file_path=processed_file_path)
    
    fetch_trending_videos_task >> data_processing_task
    
dag = trending_youtube_dag()

### load_to_sql

After we create `load_to_sqlite` function, we need to create `load_to_sqlite` task and define the sequence of the task. The `load_to_sqlite` will defined after `data_processing_task` task.

The updated dag should be look like below.

In [ ]:
@dag(dag_id='trending_youtube_dag_v1',
    default_args=default_args,
    description='A pipeline to fetch trending YouTube videos',
    start_date=datetime(2023, 5, 7, tzinfo=timezone(timedelta(hours=7))),
    schedule_interval='0 10 * * *',
    catchup=False
)
def trending_youtube_dag():
    '''
    This is youtube trending dag, we will define the task in the next section
    '''
    @task()
    def fetch_trending_videos(region_code: str, file_path: str):
        '''
        function to be used for fetching trending videos
        '''

        # Load API key from .env file
        load_dotenv("/opt/airflow/dags/.env")
        api_key = os.environ.get("YOUTUBE_API_KEY")

        # Create YouTube API client
        youtube = build("youtube", "v3", developerKey=api_key)

        # Fetch videos until max_results is reached or there are no more results
        videos_list = []
        next_page_token = ""
        while len(videos_list) < max_results and next_page_token is not None:
            # Make API request for videos
            request = youtube.videos().list(
                part="snippet,contentDetails,statistics",
                chart="mostPopular",
                regionCode=region_code,
                maxResults=50,
                pageToken=next_page_token,
            )
            response = request.execute()

            # Extract videos from response
            videos = response.get("items", [])

            # Update next_page_token for the next API request
            next_page_token = response.get("nextPageToken", None)
            
            # Extract relevant video details and append to list
            infos = {'snippet':['title', 'publishedAt', 'channelId', 'channelTitle',
                                'description', 'tags', 'thumbnails', 'categoryId', 'defaultAudioLanguage'],
                        'contentDetails':['duration', 'caption'],
                        'statistics':['viewCount', 'likeCount', 'commentCount']}
            now = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            for video in videos:
                video_details = {
                    'videoId': video["id"],
                    'trendingAt': now
                }
                
                for k in infos.keys():
                    for info in infos[k]:
                        # use try-except to handle missing info
                        try:
                            video_details[info] = video[k][info]
                        except KeyError:
                            video_details[info] = None
                            
                videos_list.append(video_details)

        # Write fetched videos data to a json file
        with open(target_file_path, "w") as f:
            json.dump(videos_list, f)
    
    @task()
    def data_processing(source_file_path: str, target_file_path: str):
        """Processes the raw data fetched from YouTube.
        
        Args:
            source_file_path: A string representing the path to the file to be processed.
            target_file_path: A string representing the path to the file to be written.
        """
        # Load the fetched videos data from the json file
        with open(source_file_path, 'r') as f:
            videos_list = json.load(f)
        
        # Load the categories dictionary from the json file
        with open('/opt/airflow/dags/categories.json', 'r') as f:
            categories = json.load(f)
        
        # Process the fetched videos data
        for video in videos_list:
            # Convert ISO 8601 duration to seconds
            video['durationSec'] = int(isodate.parse_duration(video['duration']).total_seconds()) if video['duration'] is not None else None
            del video['duration']
            
            # Convert tags list to string
            video['tags'] = ', '.join(video['tags']) if video['tags'] is not None else None
            
            # Convert categoryId to category based on categories dictionary
            video['category'] = categories.get(video['categoryId'], None) if video['categoryId'] is not None else None
            del video['categoryId']

            # Parse the thumbnail url
            video['thumbnailUrl'] = video['thumbnails'].get('standard', {}).get('url', None) if video['thumbnails'] is not None else None
            del video['thumbnails']
            
            # Convert viewCount, likeCount, and commentCount to integer
            video['viewCount'] = int(video['viewCount']) if video['viewCount'] is not None else None
            video['likeCount'] = int(video['likeCount']) if video['likeCount'] is not None else None
            video['commentCount'] = int(video['commentCount']) if video['commentCount'] is not None else None
            
            # Convert caption to boolean
            video['caption'] = True if video['caption'] == 'true' else False if video['caption'] == 'false' else None
        
        # Save the processed videos data to a new file
        with open(target_file_path, "w") as f:
            json.dump(videos_list, f)
    
    @task()
    def load_to_sqlite(source_file_path: str, table_name: str):
        """
        Loads the processed data to SQLite.
        
        Args:
            source_file_path: A string representing the path to the file to be loaded.
            table_name: A string representing the name of the table to load the data to.
        """
        
        # Load the data from the json file to sqlite
        df = pd.read_json(source_file_path)
        database = "/opt/airflow/db/airflow.db"
        conn = sqlite3.connect(database)
        
        # Append the DataFrame to the existing table if it exists, otherwise create a new table
        df.to_sql(name=table_name, con=conn, if_exists='append', index=False)
        
        conn.close()
    
        # Log the job results
        print("Done Created DB")
    
    file_path = '/opt/airflow/dags/tmp_file.json'
    fetch_trending_videos_task = fetch_trending_videos(region_code='ID', max_results=200, target_file_path=file_path)
    processed_file_path = '/opt/airflow/dags/tmp_file_processed.json'
    data_processing_task = data_processing(source_file_path=file_path, target_file_path=processed_file_path)
    load_to_sqlite_task = load_to_sqlite(source_file_path=processed_file_path, table_name='trending_videos')
    
    fetch_trending_videos_task >> data_processing_task >> load_to_sqlite_task
    
dag = trending_youtube_dag()